In [1]:
from API_call import API_call
from gamePks import *
import statsapi as mlb
import pandas as pd

from tables import *
import relationships

from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=db.db_engine)
session = Session()

Engine(sqlite://///Users/schlinkertc/code/MLB/mlb.db)


In [2]:
engine = db.db_engine
stmt="""
select
    *
from 
    matchups

"""

In [3]:
matchups = pd.read_sql(stmt,engine)

In [4]:
len(matchups['gamePk'].unique())

3953

In [5]:
stmt="""
SELECT 
    count(detailedState) as count,
    pk,
    detailedState
FROM 
    games
where 
    type = 'R'
    and 
    season = 2019
group by 
    detailedState
"""

In [6]:
df = pd.read_sql(stmt,engine)
df

,count,pk,detailedState
0,1,567304,Cancelled
1,10,567462,Completed Early: Rain
2,2419,566083,Final


In [7]:
pk = 567491
db.insert_game(pk,replace=True)

In [47]:
# query required offensive stats for Rickey's equation 
stmt=f"""
SELECT
    p.gamePk,
    p.home_away,
    sum(s.atBats) as atBats,
    sum(s.baseOnBalls)+sum(s.intentionalWalks) as BB,
    sum(s.hitByPitch) as HBP,
    sum(s.hits) as hits,
    sum(s.totalBases) as tb
FROM 
    game_player_links p
INNER JOIN 
    game_batting_stats s ON p.person_id=s.person_id
INNER JOIN 
    players on players.id=p.person_id
WHERE
    p.gamePk = {pk} AND players.primaryPosition_name != 'Pitcher'
AND 
    s.gamePk
IN
    (Select pk from games where season = 2019 and type = 'R')
group by 
    p.home_away
    """

In [48]:
engine = db.db_engine

In [56]:
offense=pd.read_sql(stmt,engine)
offense

,gamePk,home_away,atBats,BB,HBP,hits,tb
0,567491,away,4523.0,449.0,83.0,1221.0,2128.0
1,567491,home,4313.0,423.0,39.0,1170.0,2106.0


In [81]:
# query required defensive stats for Rickey's equation 
stmt=f"""
SELECT 
    p.home_away,
    players.fullName,
    sum(s.hits) as hits,
    sum(s.atBats) as atBats,
    sum(s.baseOnBalls) as walks,
    sum(s.hitBatsmen) as HBP,
    sum(s.earnedRuns) as er,
    sum(s.inningsPitched)/sum(s.gamesPitched) > 3 
        and
        sum(s.gamesStarted)/sum(s.gamesPitched) > .9
        as starter
FROM 
    game_player_links p
INNER JOIN 
    game_pitching_stats s ON p.person_id=s.person_id
INNER JOIN 
    players on players.id=p.person_id
WHERE
    p.gamePk = {pk} AND players.primaryPosition_name = 'Pitcher'
AND 
    s.gamePk
IN
    (Select pk from games where season = 2019 and type = 'R')
group by 
    players.fullName
    """

In [82]:
defense = pd.read_sql(stmt,engine)
defense

,home_away,fullName,hits,atBats,walks,HBP,er,starter
0,home,Adam Ottavino,47.0,237.0,40.0,2.0,14.0,0
1,home,Aroldis Chapman,38.0,205.0,25.0,1.0,14.0,0
2,home,CC Sabathia,112.0,422.0,39.0,3.0,59.0,1
3,home,Chad Green,66.0,267.0,19.0,6.0,32.0,0
4,home,Chance Adams,39.0,111.0,11.0,2.0,24.0,0
5,home,David Hale,39.0,148.0,7.0,1.0,13.0,0
6,away,Drew Gagnon,34.0,108.0,7.0,2.0,22.0,0
7,away,Edwin Diaz,58.0,225.0,22.0,4.0,36.0,0
8,away,Hector Santiago,42.0,140.0,22.0,0.0,25.0,0
9,home,J.A. Happ,160.0,621.0,49.0,5.0,88.0,1


In [ ]:
GamePitchingStats.game